In [ ]:
import numpy as np
import pandas as pd
dl_df = pd.read_csv("../results/dl_df.csv")
lb_df = pd.read_csv("../results/lb_df.csv")
db_df = pd.read_csv("../results/db_df.csv")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.patheffects as pe

# -------- data --------
df = lb_df.copy()

x = df["tackleOpportunities"].astype(float).to_numpy()
y = df["Average SITR"].astype(float).to_numpy()
tot = df["Total SITR"].astype(float).to_numpy()
names = df["displayName"].to_numpy()

# bubble size (robust scaling)
smin, smax = np.percentile(tot, [5, 95])
sizes = 50 + 1500 * (np.clip(tot, smin, smax) - smin) / (smax - smin + 1e-9)

# color by Average SITR
cmap = plt.get_cmap("turbo") if "turbo" in plt.colormaps() else plt.get_cmap("plasma")
norm = colors.Normalize(vmin=y.min(), vmax=y.max())

# -------- plot --------
fig = plt.figure(figsize=(11, 8))
ax = plt.gca()

sc = ax.scatter(
    x, y,
    s=sizes,
    c=norm(y), cmap=cmap,
    alpha=0.85, edgecolor="k", linewidths=0.3
)

# quadrant lines
x_med, y_med = np.median(x), np.median(y)
#ax.axvline(x_med, linestyle="--", linewidth=1, alpha=0.5)
#ax.axhline(y_med, linestyle="--", linewidth=1, alpha=0.5)

# -------- label top-4 by Total SITR --------
top4_idx = np.argsort(-tot)[:4]
xr = max(x.max() - x.min(), 1e-9)
yr = max(y.max() - y.min(), 1e-9)
dx0, dy0 = 0.06 * xr, 0.06 * yr

# force NW arrows for these two if they are in the top-4
special_nw = {"Bobby Okereke", "T.J. Edwards"}

for i in top4_idx:
    name = str(names[i])

    # label offset direction
    if name in special_nw:
        dx, dy = -abs(dx0)-3, abs(dy0)-0.2   # NW
    else:
        dx = dx0 if x[i] >= x_med else -dx0
        dy = dy0 if y[i] >= y_med else -dy0

    # arrow start position
    if name == "Pete Werner":
        # compute bubble radius in pixels from scatter size (points^2)
        r_pts = np.sqrt(sizes[i] / np.pi)              # points
        r_px  = r_pts * (fig.dpi / 72.0)               # pixels
        pad_px = 3.0                                   # tiny gap above the circle

        # transform (x,y) -> display px, add vertical offset, transform back to data
        p_disp = ax.transData.transform((x[i], y[i]-0.15))
        p_top_disp = (p_disp[0], p_disp[1] + r_px + pad_px)
        x_start, y_start = ax.transData.inverted().transform(p_top_disp)
        start_xy = (x_start, y_start)
    else:
        start_xy = (x[i], y[i])

    ann = ax.annotate(
        name,
        xy=start_xy,                           # arrow tail (starts at top of Pete's bubble)
        xytext=(x[i] + dx, y[i] + dy),         # label position
        textcoords="data",
        fontsize=9,
        ha="left", va="bottom",
        arrowprops=dict(arrowstyle="-", lw=0.9, alpha=0.9)
    )
    ann.set_path_effects([pe.withStroke(linewidth=2.5, foreground="white")])

# -------- cosmetics --------
cb = plt.colorbar(sc, ax=ax, pad=0.015)
cb.set_label("Average SITR")
cb.remove()

ax.set_xlabel("Tackle Opportunities",fontsize=18)
ax.set_ylabel("Average Solo Tackle Impact Rating",fontsize=18)
ax.set_title("Linebackers Tackle Efficiency and Volume", fontsize=18, pad=10)
ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.savefig("lb_efficiency_volume.png", dpi=300, bbox_inches="tight")
plt.show()

# Optional save:
#plt.savefig("lb_efficiency_volume.png", dpi=300, bbox_inches="tight")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.patheffects as pe

# -------- data --------
df = db_df.copy()

x = df["tackleOpportunities"].astype(float).to_numpy()
y = df["Average SITR"].astype(float).to_numpy()
tot = df["Total SITR"].astype(float).to_numpy()
names = df["displayName"].to_numpy()

# bubble size (robust scaling)
smin, smax = np.percentile(tot, [5, 95])
sizes = 50 + 1500 * (np.clip(tot, smin, smax) - smin) / (smax - smin + 1e-9)

# color by Average SITR
cmap = plt.get_cmap("turbo") if "turbo" in plt.colormaps() else plt.get_cmap("plasma")
norm = colors.Normalize(vmin=y.min(), vmax=y.max())

# -------- plot --------
fig = plt.figure(figsize=(11, 8))
ax = plt.gca()

sc = ax.scatter(
    x, y,
    s=sizes,
    c=norm(y), cmap=cmap,
    alpha=0.85, edgecolor="k", linewidths=0.3
)

# quadrant lines
x_med, y_med = np.median(x), np.median(y)
#ax.axvline(x_med, linestyle="--", linewidth=1, alpha=0.5)
#ax.axhline(y_med, linestyle="--", linewidth=1, alpha=0.5)

# -------- label top-4 by Total SITR --------
# -------- label top-4 by Total SITR --------
top4_idx = np.argsort(-tot)[:4]
xr = max(x.max() - x.min(), 1e-9)
yr = max(y.max() - y.min(), 1e-9)
dx0, dy0 = 0.06 * xr, 0.06 * yr

for i in top4_idx:
    name = str(names[i])

    # vertical offset stays based on median
    dy = dy0 if y[i] >= y_med else -dy0

    # force horizontal direction for specific players
    if name == "Anthony Brown":
        dx = -abs(dx0)-2  # go left
        dy = abs(dy0)-0.5
    elif name == "Nate Hobbs":
        dx =  abs(dx0)  # go right
        dy = abs(dy0) - 0.5
    else:
        dx = dx0 if x[i] >= x_med else -dx0  # default logic

    ann = ax.annotate(
        name,
        xy=(x[i], y[i]),
        xytext=(x[i] + dx, y[i] + dy),
        textcoords="data",
        fontsize=9,
        ha="left", va="bottom",
        arrowprops=dict(arrowstyle="-", lw=0.9, alpha=0.9)
    )
    ann.set_path_effects([pe.withStroke(linewidth=2.5, foreground="white")])

# -------- cosmetics --------
# no colorbar (per your “slide bar” ban)
ax.set_xlabel("Tackle Opportunities", fontsize=18)
ax.set_ylabel("Average Solo Tackle Impact Rating", fontsize=18)
ax.set_title("Defensive Backs Tackle Efficiency and Volume", fontsize=18, pad=10)
ax.grid(True, alpha=0.2)

plt.tight_layout()
plt.savefig("db_efficiency_volume.png", dpi=300, bbox_inches="tight")
plt.show()

# Optional save:
#plt.savefig("db_bubble_top4.png", dpi=300, bbox_inches="tight")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import FuncFormatter

# --- values ---
db_vals = db_df["Average SITR"].astype(float).to_numpy()
lb_vals = lb_df["Average SITR"].astype(float).to_numpy()

# --- shared bins ---
bins = 24
xmin = min(db_vals.min(), lb_vals.min())
xmax = max(db_vals.max(), lb_vals.max())
edges = np.linspace(xmin, xmax, bins + 1)
centers = 0.5 * (edges[:-1] + edges[1:])
bar_h = np.diff(edges)

# compare shapes; set to False for counts
density = True
h_db, _ = np.histogram(db_vals, bins=edges, density=density)
h_lb, _ = np.histogram(lb_vals, bins=edges, density=density)

# --- single colormap used for BOTH sides (same pattern) ---
cmap = plt.get_cmap("turbo") if "turbo" in plt.colormaps() else plt.get_cmap("plasma")
norm = colors.Normalize(vmin=centers.min(), vmax=centers.max())
colors_per_bin = [cmap(norm(c)) for c in centers]

fig = plt.figure(figsize=(12, 7))
ax = plt.gca()

# Bars: DB to the right, LB mirrored to the left
bars_db = ax.barh(centers,  h_db, height=bar_h, align="center",
                  edgecolor="white", linewidth=0.9, alpha=0.95, zorder=3)
bars_lb = ax.barh(centers, -h_lb, height=bar_h, align="center",
                  edgecolor="white", linewidth=0.9, alpha=0.95, zorder=3)

# Apply identical color pattern on both sides
for rect, col in zip(bars_db, colors_per_bin):
    rect.set_facecolor(col)
for rect, col in zip(bars_lb, colors_per_bin):
    rect.set_facecolor(col)

# Symmetric x-axis
m = max(h_db.max(), h_lb.max())
ax.set_xlim(-m * 1.2, m * 1.2)
ax.set_ylim(xmin, xmax)

# Bold center divider
ax.axvline(0, color="black", linewidth=5, alpha=0.95, zorder=5)

# Colored medians (red = DB, blue = LB)
med_db = float(np.median(db_vals))
med_lb = float(np.median(lb_vals))
ax.axhline(med_db, linestyle="--", linewidth=3.2, color="red",  label=f"DB median {med_db:.2f}",  zorder=4)
ax.axhline(med_lb, linestyle="-.", linewidth=3.2, color="blue", label=f"LB median {med_lb:.2f}", zorder=4)

# Huge side labels centered in each half (behind bars so they don't cover data)
ax.text(0.25, 0.50, "LB", color="blue", fontsize=48, fontweight="bold",
        ha="center", va="center", transform=ax.transAxes, alpha=0.16, zorder=1)
ax.text(0.75, 0.50, "DB", color="red",  fontsize=48, fontweight="bold",
        ha="center", va="center", transform=ax.transAxes, alpha=0.16, zorder=1)

# Bigger title/labels/ticks
ax.set_title("Solo Tackle Impact Rating Distribution— Linebackers (left) vs Defensive Backs (right)", fontsize=20, pad=12)
ax.set_xlabel("Density" if density else "Players", fontsize=18)
ax.set_ylabel("Average SITR", fontsize=18)
ax.tick_params(axis="both", labelsize=13)

# Pretty x-ticks: show absolute values
ax.xaxis.set_major_formatter(FuncFormatter(
    lambda v, pos: f"{abs(v):.2f}" if density else f"{int(abs(v))}"
))

ax.grid(True, axis="x", alpha=0.25)
ax.legend(frameon=False, loc="upper right", fontsize=12)

plt.tight_layout()
plt.savefig("stir_db_vs_lb.png", dpi=300, bbox_inches="tight")  # high-res PNG
plt.show()
